<a href="https://colab.research.google.com/github/dmi3eva/araneae/blob/main/p1_spider_analysis/04_overlapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content
%cd drive
%cd My\ Drive
%cd PhD
%cd Paper_01

/content
/content/drive
/content/drive/My Drive
/content/drive/My Drive/PhD
/content/drive/My Drive/PhD/Paper_01


# Utils

In [ ]:
DB_PATH = "datasets/spider/database"
SCHEMES_PATH = "datasets/spider/tables.json"

In [ ]:
import json
import collections
import matplotlib.pyplot as plt
import sqlite3
import os
%matplotlib inline 

In [ ]:
def execute_request(db_id, sql):
    db = os.path.join(DB_PATH, db_id, db_id + ".sqlite")
    conn = sqlite3.connect(db)
    conn.text_factory = lambda b: b.decode(errors = 'ignore')
    cursor = conn.cursor()   
    try:
        cursor.execute(sql)
        res = cursor.fetchall()
        return res
    except:
        raise ValueError

REQUEST_MASK = """SELECT \"{column}\" FROM \"{table}\"""" 
def get_values(db, table, column):
  # Возвращает значения из столбца данной таблицы
  aim_request = REQUEST_MASK.format(table=table, column=column)
  try:
    response = execute_request(db, aim_request)
    values = [str(_v[0]) for _v in list(response)]
    return values
  except ValueError:
    print()
    print(f"Problem with {column} in {table} (db = {db}). Request: {aim_request}")
    return []

db_ids = os.listdir(DB_PATH)
with open(SCHEMES_PATH) as table_file:
  schemes = json.load(table_file)

tables = {_s['db_id']: _s['table_names_original'] for _s in schemes}
sorted(list(tables.keys()))

columns = {
    _s['db_id']: {
        _table: [] for _table in _s['table_names_original']
    } for _s in schemes
}
for _scheme in schemes:
  db_id = _scheme['db_id']
  table_names = _scheme['table_names_original']
  for _column in _scheme["column_names_original"]:
    column_name = _column[1]
    table_name = table_names[_column[0]]
    if column_name == '*':
      continue
    columns[db_id][table_name].append(column_name)

def get_db_tables(db):
  return tables[db]

def get_db_columns(db, table):
  return columns[db][table]

def show_table(db, table):
  values_request = f"SELECT * FROM {table}"
  values = execute_request(db, values_request)  
  column_names = tuple(get_db_columns(db, table))
  return column_names, values

## Составление словаря
**Структура:**
```
{
  db_id: {
    token: {
      'db': set(),
      'tables': set(),
      'columns': set(),
      'values': set()
    }
  }
}
```

In [ ]:
from copy import deepcopy

In [ ]:
tokens = {}
DICT_FOR_TOKEN = {
            'db': set(),
            'tables': set(),
            'columns': set(),
            'values': set()
        }
dbs_amount = len(db_ids)
for i, _db in enumerate(db_ids):
  # print(f"{i} / {dbs_amount}")
  db_dict = {}
  db_tokens = _db.split('_')
  for _t in db_tokens:
    db_dict[_t.lower()] = deepcopy(DICT_FOR_TOKEN)    
    db_dict[_t.lower()]['db'].add(_db)
   # Обработка таблиц
  db_tables = get_db_tables(_db) 
  for _table in db_tables:
    table_tokens = _table.split('_')
    for _t in table_tokens:
      if _t.lower() not in db_dict.keys():
        db_dict[_t.lower()] = deepcopy(DICT_FOR_TOKEN) 
      db_dict[_t.lower()]['tables'].add(_table)
    # Обработка колонок
    table_columns = get_db_columns(_db, _table)
    for _column in table_columns:
       column_tokens = _column.split('_')
       for _t in column_tokens:
          if _t.lower() not in db_dict.keys():
            db_dict[_t.lower()] = deepcopy(DICT_FOR_TOKEN) 
          db_dict[_t.lower()]['columns'].add(_column)
       # Обработка значений
       column_values = get_values(_db, _table, _column)
       for _value in column_values:
         value_tokens = _value.split('_')
         for _t in value_tokens:
            if _t.lower() not in db_dict.keys():
              db_dict[_t.lower()] = deepcopy(DICT_FOR_TOKEN) 
            db_dict[_t.lower()]['values'].add(_value)

  tokens[_db] = db_dict

In [ ]:
tokens['debate']

# Анализ

In [ ]:
from pprint import pprint

In [ ]:
entities = ['db', 'tables', 'columns', 'values']
overlapping_matrix = {_e1: {_e2: 0 for _e2 in entities} for _e1 in entities}

Внутри сущности

In [ ]:
ee_json = {}
for _db in db_ids:
  ee_json[_db] = {}
  for _token in tokens[_db]:
    token_dict = tokens[_db][_token]
    for _e in entities:
      if len(token_dict[_e]) > 1:
        overlapping_matrix[_e][_e] += 1
        ee_json[_db][_e] = ee_json[_db].get(_e, []) + [str(token_dict[_e])]

In [ ]:
pprint(overlapping_matrix)

{'columns': {'columns': 2892, 'db': 0, 'tables': 0, 'values': 0},
 'db': {'columns': 0, 'db': 0, 'tables': 0, 'values': 0},
 'tables': {'columns': 0, 'db': 0, 'tables': 582, 'values': 0},
 'values': {'columns': 0, 'db': 0, 'tables': 0, 'values': 7758}}


In [ ]:
import json

In [ ]:
with open('results/in_overlapping.json', 'w') as f:
  json.dump(ee_json, f)

In [ ]:
out_json = {}
for _db in db_ids:
  out_json[_db] = {}
  for _token in tokens[_db]:
    token_dict = tokens[_db][_token]
    for _e1 in entities:
      for _e2 in entities:
        if _e1 == _e2:
          continue
        if len(token_dict[_e1]) > 0 and len(token_dict[_e2]) > 0:
           overlapping_matrix[_e1][_e2] += 1
           overlapping_matrix[_e2][_e1] += 1
           out_json[_db][f"{_e1}-{_e2}"] = out_json[_db].get(f"{_e1}-{_e2}", []) + [f"{_e1}: {str(token_dict[_e1])}, {_e2}: {str(token_dict[_e1])}"]

In [ ]:
pprint(overlapping_matrix)

{'columns': {'columns': 964, 'db': 232, 'tables': 814, 'values': 206},
 'db': {'columns': 232, 'db': 0, 'tables': 320, 'values': 90},
 'tables': {'columns': 814, 'db': 320, 'tables': 194, 'values': 74},
 'values': {'columns': 206, 'db': 90, 'tables': 74, 'values': 2586}}


In [ ]:
with open('results/out_overlapping.json', 'w') as f:
  json.dump(out_json, f)

Длинее 3 символов

In [ ]:
overlapping_matrix_3 = {_e1: {_e2: 0 for _e2 in entities} for _e1 in entities}

In [ ]:
overlapping_matrix_3

{'columns': {'columns': 0, 'db': 0, 'tables': 0, 'values': 0},
 'db': {'columns': 0, 'db': 0, 'tables': 0, 'values': 0},
 'tables': {'columns': 0, 'db': 0, 'tables': 0, 'values': 0},
 'values': {'columns': 0, 'db': 0, 'tables': 0, 'values': 0}}

In [ ]:
ee_json = {}
for _db in db_ids:
  ee_json[_db] = {}
  for _token in tokens[_db]:     
    token_dict = tokens[_db][_token]
    for _e in entities:
      if len(token_dict[_e]) > 1 and len(_token) > 3:
        overlapping_matrix_3[_e][_e] += 1
        ee_json[_db][_e] = ee_json[_db].get(_e, []) + [str(token_dict[_e])]

In [ ]:
out_json = {}
for _db in db_ids:
  out_json[_db] = {}
  for _token in tokens[_db]:
    if len(_token) <= 3:
      continue
    token_dict = tokens[_db][_token]
    for _e1 in entities:
      for _e2 in entities:
        if _e1 == _e2:
          continue
        if len(token_dict[_e1]) > 0 and len(token_dict[_e2]) > 0 and len(_token) > 3:
           overlapping_matrix_3[_e1][_e2] += 1
           overlapping_matrix_3[_e2][_e1] += 1
           out_json[_db][f"{_e1}-{_e2}"] = out_json[_db].get(f"{_e1}-{_e2}", []) + [f"{_e1}: {str(token_dict[_e1])}, {_e2}: {str(token_dict[_e1])}"]

In [ ]:
pprint(overlapping_matrix)

{'columns': {'columns': 964, 'db': 232, 'tables': 814, 'values': 206},
 'db': {'columns': 232, 'db': 0, 'tables': 320, 'values': 90},
 'tables': {'columns': 814, 'db': 320, 'tables': 194, 'values': 74},
 'values': {'columns': 206, 'db': 90, 'tables': 74, 'values': 2586}}


In [ ]:
with open('results/in_large_overlapping.json', 'w') as f:
  json.dump(ee_json, f)

In [ ]:
with open('results/out_large_overlapping.json', 'w') as f:
  json.dump(out_json, f)